In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from collections import defaultdict
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

df = pd.read_csv("filtered_1010.csv")
print(df.head())

In [ ]:
value_counts = df["true_label"].value_counts()

print(value_counts)

In [ ]:
len(df)

In [ ]:
df["sequence_id"].nunique()

In [ ]:
df["sequence_id"].describe()

In [ ]:
seq_lengths = df.groupby("sequence_id").size()
seq_lengths.mean()

In [ ]:
has_label_row = df["true_label"] != "none"
seq_has_label = (
    df.assign(has_label=has_label_row)
      .groupby("sequence_id")["has_label"]
      .any()
)
pct_sequences_with_label = seq_has_label.mean() * 100

In [ ]:
pct_sequences_with_label

In [ ]:
per_sequence["length"].describe()

In [ ]:
(per_sequence["length"] >= 16).mean() * 100

In [ ]:
with open("../heuristic_results_2.json", "r") as f:
    results = json.load(f)

In [ ]:
def extract_frame_id(path):
    return int(Path(path).stem.lstrip("0"))

In [ ]:
# Build lookup: (sequence_id, frame_id) -> true_label
label_lookup = {
    (row.sequence_id, row.frame_id): row.true_label
    for _, row in df.iterrows()
}

In [ ]:
y_true = []
y_pred = []
window_correct = []

In [ ]:
for entry in results:
    sequence_id = entry["sequence_id"]
    predicted = entry["result"]["predicted_signal"]

    frame_labels = []

    for frame_path in entry["frame_paths"]:
        frame_id = extract_frame_id(frame_path)
        label = label_lookup.get((sequence_id, frame_id))
        if label is not None:
            frame_labels.append(label)

    # Skip windows with no valid frames
    if not frame_labels:
        continue

    true_label = pd.Series(frame_labels).mode()[0]

    correct = predicted in frame_labels

    y_true.append(true_label)
    y_pred.append(predicted)
    window_correct.append(correct)

In [ ]:
overall_accuracy = sum(window_correct) / len(window_correct)
print(f"Overall accuracy: {overall_accuracy:.3f}")

In [ ]:
per_label = defaultdict(list)

for t, c in zip(y_true, window_correct):
    per_label[t].append(c)

for label, values in per_label.items():
    acc = sum(values) / len(values)
    print(f"{label:>10}: {acc:.3f} ({len(values)} windows)")

In [ ]:
labels = sorted(set(y_true) | set(y_pred))

cm = confusion_matrix(y_true, y_pred, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

print("Confusion Matrix:")
print(cm_df)

In [ ]:
print(classification_report(y_true, y_pred, labels=labels))

## Cosmos

In [ ]:
df_cosmos = pd.read_csv("../cosmos_full_predictions.csv")

In [ ]:
df_cosmos.tail()

In [ ]:
overall_acc = accuracy_score(df_cosmos["true_label"], df_cosmos["cosmos_prediction"])
print(f"Overall accuracy: {overall_acc:.4f}")

In [ ]:
df_cosmos['cosmos_prediction'].value_counts()

In [ ]:
per_label_acc = (
    df_cosmos.groupby("true_label")
      .apply(lambda x: accuracy_score(x["true_label"], x["cosmos_prediction"]))
      .sort_values(ascending=False)
)

per_label_acc

In [ ]:
labels = sorted(df_cosmos["true_label"].unique())

cm = confusion_matrix(
    df_cosmos["true_label"],
    df_cosmos["cosmos_prediction"],
    labels=labels
)

cm_df = pd.DataFrame(cm, index=labels, columns=labels)
cm_df

In [ ]:
plt.figure(figsize=(6, 6), dpi=300)
plt.imshow(cm, interpolation="nearest", cmap='Reds')
plt.title("Confusion Matrix")
plt.colorbar()

tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)

plt.xlabel("Predicted label")
plt.ylabel("True label")

for i in range(len(labels)):
    for j in range(len(labels)):
        plt.text(j, i, cm[i, j], ha="center", va="center")

plt.tight_layout()
plt.show()